In [2]:
# ✅ Step 1: Clone fresh copy
!rm -rf agrix_beta_2025
!git clone https://github.com/smwaingeni-ai/agrix_beta_2025.git

# ✅ Step 2: Import libraries
import pandas as pd
from datetime import datetime

# ✅ Step 3: Load and clean the CSV
rules_path = "/content/agrix_beta_2025/assets/data/livestock_rules.csv"

try:
    rules_df = pd.read_csv(rules_path, encoding='ISO-8859-1', quotechar='"')
    rules_df.columns = rules_df.columns.str.strip().str.replace('\ufeff', '').str.lower()

    required_cols = ['symptom', 'disease', 'treatment', 'species', 'severity', 'likelihoodscore', 'sampleimage']
    for col in required_cols:
        if col not in rules_df.columns:
            raise KeyError(f"❌ Missing column: '{col}'")

    print("✅ Columns loaded:", list(rules_df.columns))
except Exception as e:
    print(f"❌ Failed to load livestock_rules.csv: {e}")
    raise

# ✅ Step 4: Build symptom rule dictionary
symptom_to_diagnosis = {}

for _, row in rules_df.iterrows():
    key = row['symptom'].strip().lower()
    symptom_to_diagnosis[key] = {
        "disease": row['disease'],
        "treatment": row['treatment'],
        "species": row['species'],
        "severity": row['severity'],
        "likelihood": float(row['likelihoodscore']),
        "image": row['sampleimage']
    }

# ✅ Step 5: Partial match logic with likelihood sorting
def find_partial_matches(input_symptom, rule_dict):
    input_symptom = input_symptom.lower().strip()
    matches = []

    for symptom_phrase, details in rule_dict.items():
        if input_symptom in symptom_phrase:
            matches.append((symptom_phrase, details))

    # 🔽 Sort matches by likelihood descending
    matches.sort(key=lambda x: x[1]['likelihood'], reverse=True)
    return matches

# ✅ Step 6: Simulated user input (enter one word or phrase)
user_input_symptoms = [
    "fever",       # should match multiple
    "diarrhea",    # should match Coccidiosis
    "maggots",     # should match Flystrike
    "hoof",        # should match Footrot
    "nasal",       # should match CBPP & Pink Eye
    "unknown"      # should show no match
]

# ✅ Step 7: Run diagnoses
print("\n🩺 Diagnosis Results (Partial Match + Ranking):\n")
diagnosis_log = []

for input_term in user_input_symptoms:
    matches = find_partial_matches(input_term, symptom_to_diagnosis)

    if matches:
        for matched_symptom, info in matches:
            print(f"🔍 Matched Phrase: '{matched_symptom}' for input: '{input_term}'")
            print(f"➡️ Disease: {info['disease']}")
            print(f"💊 Treatment: {info['treatment']}")
            print(f"🐄 Species: {info['species']}")
            print(f"⚠️ Severity: {info['severity']}")
            print(f"📊 Likelihood Score: {info['likelihood']}")
            print(f"🖼️ Image: {info['image']}\n")

            diagnosis_log.append({
                "timestamp": datetime.now().isoformat(),
                "input_term": input_term,
                "matched_symptom": matched_symptom,
                "disease": info["disease"],
                "treatment": info["treatment"],
                "species": info["species"],
                "severity": info["severity"],
                "likelihood": info["likelihood"],
                "image": info["image"]
            })
    else:
        print(f"⚠️ No match found for input: '{input_term}'\n")

# ✅ Step 8: Save log if results found
if diagnosis_log:
    df_log = pd.DataFrame(diagnosis_log)
    log_path = "livestock_diagnosis_log.csv"
    df_log.to_csv(log_path, index=False)
    print(f"📝 Diagnosis log saved to: {log_path}")
else:
    print("📭 No matches found — no log saved.")


Cloning into 'agrix_beta_2025'...
remote: Enumerating objects: 1980, done.
remote: Counting objects: 100% (326/326), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 1980 (delta 118), reused 37 (delta 37), pack-reused 1654 (from 1)
Receiving objects: 100% (1980/1980), 75.15 MiB | 22.43 MiB/s, done.
Resolving deltas: 100% (925/925), done.
✅ Columns loaded: ['symptom', 'disease', 'treatment', 'species', 'severity', 'likelihoodscore', 'sampleimage']

🩺 Diagnosis Results (Partial Match + Ranking):

🔍 Matched Phrase: 'shaking and high fever' for input: 'fever'
➡️ Disease: Rift Valley Fever
💊 Treatment: No treatment â supportive care only
🐄 Species: Cattle
⚠️ Severity: Severe
📊 Likelihood Score: 0.93
🖼️ Image: rvf.jpg

🔍 Matched Phrase: 'eye discharge and fever' for input: 'fever'
➡️ Disease: New Castle Disease
💊 Treatment: Apply eye drops and vaccinate survivors
🐄 Species: Poultry
⚠️ Severity: Severe
📊 Likelihood Score: 0.9
🖼️ Image: newcastle.jpg

🔍 Matched Phrase: 